# Importando as Bibliotecas

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window as W

# Criando o Spark Session

In [2]:
spark = (SparkSession.builder
        .master("local")
        .appName("Pyspark_RJ")
        .config("spark.ui.port", "4050")
        .getOrCreate()
        )

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
spark

# Extração de dados - Rio de Janeiro

In [4]:
df = spark.read.parquet('gs://projeto_final_2021/pandas_to_parquet/RJ_parquet/df_completo.snappy')

In [5]:
# Visualização do schema e tipo de dados
df.printSchema()

root
 |-- Matrícula: integer (nullable = true)
 |-- Nome: string (nullable = true)
 |-- Data_Admissão: timestamp (nullable = true)
 |-- Cargo: string (nullable = true)
 |-- Unidade: string (nullable = true)
 |-- Local_de_Trabalho: string (nullable = true)
 |-- Número_do_ATO: string (nullable = true)
 |-- Data_Publicação: timestamp (nullable = true)
 |-- Concursado: string (nullable = true)
 |-- Comissionado: string (nullable = true)
 |-- Função_de_Confiança: string (nullable = true)
 |-- Vencimento_Bruto: float (nullable = true)
 |-- Desconto_de_Imposto_IR: float (nullable = true)
 |-- Desconto_Previdenciário: float (nullable = true)
 |-- Outros_Descontos: float (nullable = true)
 |-- Vencimento_Líquido: float (nullable = true)
 |-- Mês: byte (nullable = true)
 |-- ANO: short (nullable = true)
 |-- Tipo_de_Folha: string (nullable = true)
 |-- Regime: string (nullable = true)
 |-- Contratado: string (nullable = true)
 |-- Estado: string (nullable = true)



In [6]:
df.show()

+---------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------+-------------------+----------+------------+-------------------+----------------+----------------------+-----------------------+----------------+------------------+---+----+-------------+--------------------+----------+------+
|Matrícula|                Nome|      Data_Admissão|               Cargo|             Unidade|   Local_de_Trabalho|Número_do_ATO|    Data_Publicação|Concursado|Comissionado|Função_de_Confiança|Vencimento_Bruto|Desconto_de_Imposto_IR|Desconto_Previdenciário|Outros_Descontos|Vencimento_Líquido|Mês| ANO|Tipo_de_Folha|              Regime|Contratado|Estado|
+---------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------+-------------------+----------+------------+-------------------+----------------+----------------------+-----------------------+----------------+--------------

In [7]:
#verificar nomes das colunas 
df.schema.names

['Matrícula',
 'Nome',
 'Data_Admissão',
 'Cargo',
 'Unidade',
 'Local_de_Trabalho',
 'Número_do_ATO',
 'Data_Publicação',
 'Concursado',
 'Comissionado',
 'Função_de_Confiança',
 'Vencimento_Bruto',
 'Desconto_de_Imposto_IR',
 'Desconto_Previdenciário',
 'Outros_Descontos',
 'Vencimento_Líquido',
 'Mês',
 'ANO',
 'Tipo_de_Folha',
 'Regime',
 'Contratado',
 'Estado']

In [8]:
#Alterar nome da coluna
df = (df.withColumnRenamed('Mês', 'mes')
  .withColumnRenamed('ANO', 'ano')
  .withColumnRenamed('Cargo', 'cargo')
  .withColumnRenamed('Vencimento_Bruto', 'salarioBruto')
  .withColumnRenamed('Vencimento_Líquido', 'salarioLiquido')
  .withColumnRenamed('Estado', 'estado')
  .withColumnRenamed('Contratado', 'contratado')
  .withColumnRenamed('Nome', 'nome')

)
df.show()

+---------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------+-------------------+----------+------------+-------------------+------------+----------------------+-----------------------+----------------+--------------+---+----+-------------+--------------------+----------+------+
|Matrícula|                nome|      Data_Admissão|               cargo|             Unidade|   Local_de_Trabalho|Número_do_ATO|    Data_Publicação|Concursado|Comissionado|Função_de_Confiança|salarioBruto|Desconto_de_Imposto_IR|Desconto_Previdenciário|Outros_Descontos|salarioLiquido|mes| ano|Tipo_de_Folha|              Regime|contratado|estado|
+---------+--------------------+-------------------+--------------------+--------------------+--------------------+-------------+-------------------+----------+------------+-------------------+------------+----------------------+-----------------------+----------------+--------------+---+----+----------

In [9]:
# Inclusão de coluna
value = 'ATIVO'

df = df.withColumn('situacao', lit(value).cast('string'))

In [10]:
# Remoção de colunas 

coletar_colunas = {'mes','ano','cargo','situacao','salarioLiquido','nome', 'estado'}

remover_colunas = list(set(df.schema.names).difference(coletar_colunas))

df_rj = df.drop(*remover_colunas)

In [11]:
# Colocar as colunas em ordem correta 
coletar_colunas = ('mes','ano','nome','cargo','situacao','salarioLiquido', 'estado')
df_rj = df_rj.select(*coletar_colunas)

In [12]:
df_rj.show()

+---+----+--------------------+--------------------+--------+--------------+------+
|mes| ano|                nome|               cargo|situacao|salarioLiquido|estado|
+---+----+--------------------+--------------------+--------+--------------+------+
|  8|2021|ABEIGAIR DE ALENC...|    ENFERMEIRO - ESF|   ATIVO|       4281.61|    RJ|
|  8|2021|ADAIL JULIO DO NA...|    GUARDA MUNICIPAL|   ATIVO|       4013.84|    RJ|
|  8|2021|ADALBERTO CURVELO...|SUBSECRETÁRIO DE ...|   ATIVO|       3799.06|    RJ|
|  8|2021|ADEILTON ALVES DE...|    GUARDA MUNICIPAL|   ATIVO|       1220.18|    RJ|
|  8|2021|ADEMILTON DE OLIV...|  TRABALHADOR BRAÇAL|   ATIVO|       2226.98|    RJ|
|  8|2021|ADERLANE  CRISTIN...|TECNICO EM ENFERM...|   ATIVO|       1372.31|    RJ|
|  8|2021|ADILSON MACHADO M...|AUXILIAR DE ARTIFICE|   ATIVO|       2511.15|    RJ|
|  8|2021|ADJANE BASILIO FA...|         PROFESSOR I|   ATIVO|        182.93|    RJ|
|  8|2021|ADRIANA APARECIDA...|ASSESSOR DE INTEG...|   ATIVO|       1973.87|

In [13]:
df_rj.filter(df_rj.estado == "RJ").filter(df_rj.ano == 2021).filter(df_rj.mes == 2).show()

+---+----+--------------------+--------------------+--------+--------------+------+
|mes| ano|                nome|               cargo|situacao|salarioLiquido|estado|
+---+----+--------------------+--------------------+--------+--------------+------+
|  2|2021|ABEIGAIR DE ALENC...|    ENFERMEIRO - ESF|   ATIVO|       5672.84|    RJ|
|  2|2021|ADAIL JULIO DO NA...|    GUARDA MUNICIPAL|   ATIVO|        7751.5|    RJ|
|  2|2021|ADALBERTO CURVELO...|SUBSECRETÁRIO DE ...|   ATIVO|       3799.06|    RJ|
|  2|2021|ADEILTON ALVES DE...|    GUARDA MUNICIPAL|   ATIVO|       1980.26|    RJ|
|  2|2021|ADEMILTON DE OLIV...|  TRABALHADOR BRAÇAL|   ATIVO|       1986.65|    RJ|
|  2|2021|ADILSON MACHADO M...|AUXILIAR DE ARTIFICE|   ATIVO|       3046.69|    RJ|
|  2|2021|ADJANE BASILIO FA...|         PROFESSOR I|   ATIVO|       1172.24|    RJ|
|  2|2021|ADRIANA APARECIDA...|AUXILIAR OPERACIO...|   ATIVO|        1696.2|    RJ|
|  2|2021|ADRIANA CHAGAS DO...|AUXILIAR DE ENFER...|   ATIVO|       1772.16|

In [14]:
#Salvar dados 
(df_rj.write.format("parquet")
.option("header", "true")
.option("inferschema", "true")
.save("gs://projeto_final_2021/parquet_to_bq/RJ")
)


AnalysisException: path gs://projeto_final_2021/parquet_to_bq/RJ already exists.

In [ ]:
df.filter(df.estado == "RJ").filter(df.ano == 2021).filter(df.mes == 2).show()